In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
file_path = "/content/utilization.json"
df = spark.read.format("json").load(file_path)

In [3]:
df.createOrReplaceTempView("utilization")

In [4]:
spark.sql("SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
          FROM utilization \
          GROUP BY server_id" ).show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      112|                0.52|                0.92|    0.11528867845082576|
|      113|                0.58|                0.98|    0.11544345150353694|
|      126|                0.48|                0.88|    0.11542612970702058|
|      110|                0.35|                0.75|    0.11533251724450215|
|      119|                0.22|                0.62|    0.11516031929842008|
|      116|                 0.3|                 0.7|    0.11506079722349302|
|      124|                0.24|                0.64|    0.11579377614906383|
|      107|                0.45|                0.85|    0.11597417369783877|
|      103|                0.56|                0.96|    0.11617507884178278|
|      114|                0.33|                0.73|    0.11510

In [7]:
sql_window = spark.sql("SELECT event_datetime, server_id, cpu_utilization, \
                      avg(cpu_utilization) OVER (PARTITION BY server_id) avg_server_util \
                      FROM utilization")

In [8]:
sql_window.show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:19|      103|           0.64|0.7614389999999969|
|03/05/2019 08:11:19|      103|           0.76|0.7614389999999969|
|03/05/2019 08:16:19|      103|           0.61|0.7614389999999969|
|03/05/2019 08:21:19|      103|           0.95|0.7614389999999969|
|03/05/2019 08:26:19|      103|           0.87|0.7614389999999969|
|03/05/2019 08:31:19|      103|           0.61|0.7614389999999969|
|03/05/2019 08:36:19|      103|            0.8|0.7614389999999969|
|03/05/2019 08:41:19|      103|           0.73|0.7614389999999969|
|03/05/2019 08:46:19|      103|           0.85|0.7614389999999969|
|03/05/2019 08:51:19|      103|           0.79|0.7614389999999969|
|03/05/2019 08:56:19|      103|           0.57|0.7614389999999969|
|03/05/2019 09:01:19|      103|           0.72|0.7614389999999

# Using Window Functions

In [14]:
sql_window2 = spark.sql("SELECT event_datetime, server_id, cpu_utilization, \
                      avg(cpu_utilization) OVER (PARTITION BY server_id) avg_server_util, \
                      cpu_utilization - avg(cpu_utilization) OVER (PARTITION BY server_id) delta_server_util \
                      FROM utilization")


In [15]:
sql_window2.show()

+-------------------+---------+---------------+------------------+--------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|   delta_server_util|
+-------------------+---------+---------------+------------------+--------------------+
|03/05/2019 08:06:19|      103|           0.64|0.7614389999999969|-0.12143899999999686|
|03/05/2019 08:11:19|      103|           0.76|0.7614389999999969|-0.00143899999999...|
|03/05/2019 08:16:19|      103|           0.61|0.7614389999999969|-0.15143899999999688|
|03/05/2019 08:21:19|      103|           0.95|0.7614389999999969|  0.1885610000000031|
|03/05/2019 08:26:19|      103|           0.87|0.7614389999999969| 0.10856100000000313|
|03/05/2019 08:31:19|      103|           0.61|0.7614389999999969|-0.15143899999999688|
|03/05/2019 08:36:19|      103|            0.8|0.7614389999999969|0.038561000000003176|
|03/05/2019 08:41:19|      103|           0.73|0.7614389999999969|-0.03143899999999...|
|03/05/2019 08:46:19|      103| 

In [19]:
# The ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING window function considers three rows:
# - The previous row (1 row before the current row)
# - The current row
# - The next row (1 row after the current row) In this case, it applies to the
# cpu_utilization column, meaning the AVG calculation would consider these three rows


sql_window3 = spark.sql("SELECT event_datetime, server_id, cpu_utilization, \
                        avg(cpu_utilization) OVER (PARTITION BY server_id ORDER BY event_datetime \
                                             ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) avg_server_util \
                        FROM utilization")

In [18]:
sql_window3.show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:19|      103|           0.64|               0.7|
|03/05/2019 08:11:19|      103|           0.76|0.6699999999999999|
|03/05/2019 08:16:19|      103|           0.61|0.7733333333333334|
|03/05/2019 08:21:19|      103|           0.95|              0.81|
|03/05/2019 08:26:19|      103|           0.87|0.8099999999999999|
|03/05/2019 08:31:19|      103|           0.61|0.7600000000000001|
|03/05/2019 08:36:19|      103|            0.8|0.7133333333333334|
|03/05/2019 08:41:19|      103|           0.73|0.7933333333333333|
|03/05/2019 08:46:19|      103|           0.85|              0.79|
|03/05/2019 08:51:19|      103|           0.79|0.7366666666666667|
|03/05/2019 08:56:19|      103|           0.57|0.6933333333333334|
|03/05/2019 09:01:19|      103|           0.72|0.6633333333333